In [1]:
import autograd.numpy as np
from autograd import grad
from autograd.extend import primitive, defvjp
import numpy as npf
import sys,time
sys.path.append('../')
import use_autograd
use_autograd.use = 1
import rcwa
from fft_funs import get_conv
from utils import test_grad

In [2]:
reload(rcwa)
nG = 21
# lattice vector
Lx = .1
Ly = .1
L1 = [Lx,0.]
L2 = [0.,Ly]

# frequency and angles
freq = 1.
theta = np.pi/4
phi = 0.

# now consider three layers: vacuum + patterned + vacuum
epsuniform1 = 2.
epsuniform3 = 1.
thick1 = 1.
thick2 = 2.
thick3 = 1.

# for patterned layer, eps = epsbkg + dof * epsdiff
epsbkg = 1.
epsdiff = -2+1j

# discretization for patterend layer
Nx = 10
Ny = 10

# setup RCWA
obj = rcwa.RCWA_obj(nG,L1,L2,freq,theta,phi)
obj.Add_LayerUniform(thick1,epsuniform1)
obj.Add_LayerGrid(thick2,epsdiff,epsbkg,Nx,Ny)
obj.Add_LayerUniform(thick3,epsuniform3)
obj.Init_Setup(Gmethod=0)


# planewave excitation
p_amp = 0.
s_amp = 1.
p_phase = 0.
s_phase = 0.

obj.MakeExcitationPlanewave(p_amp,p_phase,s_amp,s_phase,order = 0)

('Total nG = ', 21)


In [3]:
def fun_volume(x):
    obj.GridLayer_getDOF(x)
    y = np.reshape(x,(Nx,Ny))
    #Mv = get_conv(1./Nx/Ny,y,obj.G)
    Mv = np.ones((obj.nG,obj.nG))
    
    val = obj.Volume_integral(1,Mv,Mv,Mv,normalize=1)
    #ai,bi = obj.GetAmplitudes(1,0.)
    #ai = np.dot(obj.phi_list[1],ai)
    #val = np.sum(ai)
    return np.real(val)

In [4]:
grad_fun = grad(fun_volume)

x = np.random.random(Nx*Ny)
test_grad(fun_volume,grad_fun,x,1e-5,4)

Finite difference =  0.018437789199576837 , Auto =  0.018437772848196067


In [ ]:
def fun(x):
    obj.GridLayer_getDOF(x)
    R,T = obj.RT_Solve()
    return T

In [ ]:
grad_fun = grad(fun)

x = np.random.random(Nx*Ny)
test_grad(fun,grad_fun,x,1e-4,0)